# Generalised Linear Models

### [Neil D. Lawrence](http://inverseprobability.com), University of

Cambridge

### 2025-09-10

$$
$$

<!-- Do not edit this file locally. -->
<!-- Do not edit this file locally. -->
<!---->
<!-- Do not edit this file locally. -->
<!-- Do not edit this file locally. -->
<!-- The last names to be defined. Should be defined entirely in terms of macros from above-->
<!--

-->

## ML Foundations Course Notebook Setup

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_mlfc/includes/mlfc-notebook-setup.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_mlfc/includes/mlfc-notebook-setup.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

We install some bespoke codes for creating and saving plots as well as
loading data sets.

In [ ]:
%%capture
%pip install notutils
%pip install pods
%pip install mlai

In [ ]:
import notutils
import pods
import mlai
import mlai.plot as plot

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})

<!--setupplotcode{import seaborn as sns
sns.set_style('darkgrid')
sns.set_context('paper')
sns.set_palette('colorblind')}-->

## Review

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/generalised-linear-models.gpp.markdown" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/generalised-linear-models.gpp.markdown', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

We introduced machine learning as a way to extract knowledge from data
to make predictions through a prediction function and an objective
function. We looked at a simple example of predicting whether someone
would buy a jumper based on their age and latitude, *using logistic
regression* to model the log-odds of purchase. This highlighted how
machine learning can codify predictions through mathematical functions.
This is an example of a broader approach known as *generalised linear
models*.

When taking a probabilistic approach to supervised learning we’re
interested in predicting a class label, $y_i$, given an input,
$\mathbf{ x}_i$. That’s represented probabilisticially as
$p(y_i|\mathbf{ x}_i)$. We can derive this conditional distribution
through either (1) modelling the joint distribution,
$p(\mathbf{ y}, \mathbf{X})$ and then dividing by the marginal
distribution of the inputs, $p(\mathbf{X})$ , or (2) focusing
specifically on modeling the conditional density,
$p(\mathbf{ y}|\mathbf{X})$, that directly answers our prediction
question. In the *generalised linear model* we choose the second
approach.

As we move to generalised linear models like logistic regression, we’ll
see how directly modeling the conditional density
$p(\mathbf{ y}|\mathbf{X})$ can provide more flexibility in our modeling
assumptions, while still allowing us to make the specific predictions we
need.

## Logistic Regression

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_ml/includes/logistic-regression-intro.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_ml/includes/logistic-regression-intro.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

A logistic regression is an approach to classification which extends the
linear regression models we’ve already explored. Rather than modeling
the output of the function directly the assumption is that we model the
*log-odds* with the basis functions.

The [odds](http://en.wikipedia.org/wiki/Odds) are defined as the ratio
of the probability of a positive outcome, to the probability of a
negative outcome. Just as we saw in our jumper (sweater) example where
$$
\log \frac{p(\text{bought})}{p(\text{not bought})} = w_0 + w_1 \text{age} + w_2 \text{latitude}
$$ If the probability of a positive outcome is denoted by $\pi$, then
the odds are computed as $\frac{\pi}{1-\pi}$.

Odds are widely used by
[bookmakers](http://en.wikipedia.org/wiki/Bookmaker) in gambling,
although a bookmakers odds won’t normalise: i.e. if you look at the
equivalent probabilities, and sum over the probability of all outcomes
the bookmakers are considering, then you won’t get one. This is how the
bookmaker makes a profit. Because a probability is always between zero
and one, the odds are always between $0$ and $\infty$. If the positive
outcome is unlikely the odds are close to zero, if it is very likely
then the odds become close to infinite. Taking the logarithm of the odds
maps the odds from the positive half space to being across the entire
real line. Odds that were between 0 and 1 (where the negative outcome
was more likely) are mapped to the range between $-\infty$ and $0$. Odds
that are greater than 1 are mapped to the range between $0$ and
$\infty$. Considering the log odds therefore takes a number between 0
and 1 (the probability of positive outcome) and maps it to the entire
real line. The function that does this is known as the [logit
function](http://en.wikipedia.org/wiki/Logit),
$g(p_i) = \log\frac{p_i}{1-p_i}$. This function is known as a *link
function*.

For a standard regression we take, $$
f(\mathbf{ x}) = \mathbf{ w}^\top
\boldsymbol{ \phi}(\mathbf{ x}),
$$ if we want to perform classification we perform a logistic
regression. $$
\log \frac{\pi}{(1-\pi)} = \mathbf{ w}^\top
\boldsymbol{ \phi}(\mathbf{ x})
$$ where the odds ratio between the positive class and the negative
class is given by $$
\frac{\pi}{(1-\pi)}
$$ The odds can never be negative, but can take any value from 0 to
$\infty$. We have defined the link function as taking the form
$g^{-1}(\cdot)$ implying that the inverse link function is given by
$g(\cdot)$. Since we have defined, $$
g(\pi) =
\mathbf{ w}^\top \boldsymbol{ \phi}(\mathbf{ x})
$$ we can write $\pi$ in terms of the *inverse link* function,
$h(\cdot)$ as $$
\pi = h(\mathbf{ w}^\top
\boldsymbol{ \phi}(\mathbf{ x})).
$$

In [ ]:
import mlai.plot as plot

In [ ]:
plot.logistic('./ml')

We’ll define our prediction, objective and gradient functions below. But
before we start, we need to define a basis function for our model. Let’s
start with the linear basis.

In [ ]:
import numpy as np

In [ ]:
import mlai
import inspect
file_path = inspect.getfile(mlai.linear)

In [ ]:
%load -s linear {file_path}

### Sigmoid Function

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_ml/includes/sigmoid-function.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_ml/includes/sigmoid-function.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

In [ ]:
import mlai.plot as plot

In [ ]:
plot.logistic('./ml')

<img src="https://mlatcl.github.io/mlfc/./slides/diagrams//ml/logistic.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>The logistic function.</i>

The function has this characeristic ‘s’-shape (from where the term
sigmoid, as in sigma, comes from). It also takes the input from the
entire real line and ‘squashes’ it into an output that is between zero
and one. For this reason it is sometimes also called a ‘squashing
function.’

The sigmoid comes from the inverting the odds ratio, $$
\frac{\pi}{(1-\pi)}
$$ where $\pi$ is the probability of a positive outcome and $1-\pi$ is
the probability of a negative outcome

## Prediction Function

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_ml/includes/logistic-regression-prediction-function.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_ml/includes/logistic-regression-prediction-function.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

Now we have the basis function let’s define the prediction function.

In [ ]:
import numpy as np

In [ ]:
def predict(w, x, basis=mlai.linear, **kwargs):
    "Generates the prediction function and the basis matrix."
    Phi = basis(x, **kwargs)
    f = np.dot(Phi, w)
    return 1./(1+np.exp(-f)), Phi

This inverse of the link function is known as the
[logistic](http://en.wikipedia.org/wiki/Logistic_function) (thus the
name logistic regression) or sometimes it is called the sigmoid
function. For a particular value of the input to the link function,
$f_i = \mathbf{ w}^\top \boldsymbol{ \phi}(\mathbf{ x}_i)$ we can plot
the value of the inverse link function as below.

By replacing the inverse link with the sigmoid we can write $\pi$ as a
function of the input and the parameter vector as, $$
\pi(\mathbf{ x},\mathbf{ w}) = \frac{1}{1+\exp\left(-\mathbf{ w}^\top \boldsymbol{ \phi}(\mathbf{ x})\right)}.
$$ The process for logistic regression is as follows. Compute the output
of a standard linear basis function composition
($\mathbf{ w}^\top \boldsymbol{ \phi}(\mathbf{ x})$, as we did for
linear regression) and then apply the inverse link function,
$g(\mathbf{ w}^\top \boldsymbol{ \phi}(\mathbf{ x}))$. In logistic
regression this involves *squashing* it with the logistic (or sigmoid)
function. Use this value, which now has an interpretation as a
*probability* in a Bernoulli distribution to form the likelihood. Then
we can assume conditional independence of each data point given the
parameters and develop a likelihod for the entire data set.

The Bernoulli likelihood is of the form, $$
P(y_i|\mathbf{ w}, \mathbf{ x}) =
\pi_i^{y_i} (1-\pi_i)^{1-y_i}
$$ which we can think of as clever trick for mathematically switching
between two probabilities if we were to write it as code it would be
better described as

``` python
def bernoulli(x, y, pi):
    if y == 1:
        return pi(x)
    else:
        return 1-pi(x)
```

but writing it mathematically makes it easier to write our objective
function within a single mathematical equation.

## Objective Function

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_ml/includes/logistic-regression-maximum-likelihood.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_ml/includes/logistic-regression-maximum-likelihood.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

As normal, we would like to minimise this objective. This can be done by
differentiating with respect to the parameters of our prediction
function, $\pi(\mathbf{ x};\mathbf{ w})$, for optimisation. The gradient
of the likelihood with respect to $\pi(\mathbf{ x};\mathbf{ w})$ is of
the form, $$
\frac{\text{d}E(\mathbf{ w})}{\text{d}\mathbf{ w}} = -\sum_{i=1}^n
\frac{y_i}{h\left(\mathbf{ w}^\top
\boldsymbol{ \phi}(\mathbf{ x})\right)}\frac{\text{d}h(f_i)}{\text{d}f_i}
\boldsymbol{ \phi}(\mathbf{ x}_i) +  \sum_{i=1}^n
\frac{1-y_i}{1-h\left(\mathbf{ w}^\top
\boldsymbol{ \phi}(\mathbf{ x})\right)}\frac{\text{d}h(f_i)}{\text{d}f_i}
\boldsymbol{ \phi}(\mathbf{ x}_i)
$$ where we used the chain rule to develop the derivative in terms of
$\frac{\text{d}h(f_i)}{\text{d}f_i}$, which is the gradient of the
inverse link function (in our case the gradient of the sigmoid
function).

So the objective function now depends on the gradient of the inverse
link function, as well as the likelihood depends on the gradient of the
inverse link function, as well as the gradient of the log likelihood,
and naturally the gradient of the argument of the inverse link function
with respect to the parameters, which is simply
$\boldsymbol{ \phi}(\mathbf{ x}_i)$.

The only missing term is the gradient of the inverse link function. For
the sigmoid squashing function we have, $$\begin{align*}
h(f_i) &= \frac{1}{1+\exp(-f_i)}\\
&=(1+\exp(-f_i))^{-1}
\end{align*}$$ and the gradient can be computed as $$\begin{align*}
\frac{\text{d}h(f_i)}{\text{d} f_i} & =
\exp(-f_i)(1+\exp(-f_i))^{-2}\\
& = \frac{1}{1+\exp(-f_i)}
\frac{\exp(-f_i)}{1+\exp(-f_i)} \\
& = h(f_i) (1-g(f_i))
\end{align*}$$ so the full gradient can be written down as $$
\frac{\text{d}E(\mathbf{ w})}{\text{d}\mathbf{ w}} = -\sum_{i=1}^n
y_i\left(1-h\left(\mathbf{ w}^\top \boldsymbol{ \phi}(\mathbf{ x})\right)\right)
\boldsymbol{ \phi}(\mathbf{ x}_i) +  \sum_{i=1}^n
(1-y_i)\left(h\left(\mathbf{ w}^\top \boldsymbol{ \phi}(\mathbf{ x})\right)\right)
\boldsymbol{ \phi}(\mathbf{ x}_i).
$$

In [ ]:
import numpy as np

In [ ]:
def gradient(h, Phi, y):
    "Generates the gradient of the parameter vector."
    labs = np.asarray(y, dtype=float).flatten()
    posind = np.where(labs==1)
    dw = -(Phi[posind]*(1-g[posind])).sum(0)
    negind = np.where(labs==0 )
    dw += (Phi[negind]*h[negind]).sum(0)
    return dw[:, None]

## Optimisation of the Function

Reorganising the gradient to find a stationary point of the function
with respect to the parameters $\mathbf{ w}$ turns out to be impossible.
Optimisation has to proceed by *numerical methods*. Options include the
multidimensional variant of [Newton’s
method](http://en.wikipedia.org/wiki/Newton%27s_method) or [gradient
based optimisation
methods](http://en.wikipedia.org/wiki/Gradient_method) like we used for
optimising matrix factorisation for the movie recommender system. We
recall from matrix factorisation that, for large data, *stochastic
gradient descent* or the Robbins Munro (Robbins and Monro, 1951)
optimisation procedure worked best for function minimisation.

-   Can’t find a stationary point of the objective function
    analytically.
-   Optimisation has to proceed by *numerical methods*.
    -   [Newton’s
        method](http://en.wikipedia.org/wiki/Newton%27s_method) or
    -   [gradient based optimisation
        methods](http://en.wikipedia.org/wiki/Gradient_method)
    -   *Iterative Reweighted Least Squares*
-   Similarly to matrix factorisation, for large data *stochastic
    gradient descent* (Robbins Munro (Robbins and Monro, 1951)
    optimisation procedure) works well.

<!--include{_datasets/includes/classification-toy-data.md}

Now from the toy data we create design matrices with a leading column of ones (an Eins column)

:::

::: {.cell .code}

```{.python}
import numpy as np

In [ ]:
phi_plus = np.hstack([np.ones((x_plus.shape[0], 1)), x_plus])
phi_minus = np.hstack([np.ones((x_minus.shape[0], 1)), x_minus])

## Logistic Regression vs Perceptron

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_ml/includes/logistic-regression-perceptron.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_ml/includes/logistic-regression-perceptron.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

We already looked at the Perceptron, where the prediction function was
given by $$
\mappingFunction(\mappingVector, \inputVector_i) = \text{sign}(\designVector_i^\top \mappingVector)
$$ and the sign of the output gave us the class, where we had a positive
and negative class.

In logistic regression, the prediction function gives us the probability
of positive class, $$
\mappingFunction(\mappingVector, \inputVector) = \pi_i = \transformationFunction(\designVector_i^\top \mappingVector)
$$ so the two are closely related. We can think of the sigmoid as
providing a “soft decision” whereas the sign provides a hard decision.

Let’s make a more detailed study by considering a *stochastic gradient
descent* algorithm for the logistic regression of the negative log
likelihood. We’ll compare that update to the Perceptron update.

We can think of the Perceptron update as randomly sampling a data point,
$i$, checking its prediction. If the predicted class doesn’t match the
true class, $y_i$, then we update the weights by subtracting
$\designVector_i$ if the prediction is positive, and adding
$\designVector_i$ if the prediction is negative. Using
$\dataScalar_i\in\{0,1\}$ to match logistic regression, a compact update
is $$
 \mappingVector_\text{new} \leftarrow \mappingVector_\text{old} - \eta(\heaviside(\designVector_i^\top \mappingVector) (1-\dataScalar_i) \designVector_i - (1-\heaviside(\designVector_i^\top \mappingVector)) \dataScalar_i \designVector_i
$$

The gradient of the negative log-likelihood of logistic regression is $$
\frac{\text{d}\errorFunction(\mappingVector)}{\text{d}\mappingVector} = -\sum_{i=1}^\numData
\dataScalar_i\left(1-\transformationFunction\left(\mappingVector^\top \designVector_i\right)\right)
\designVector_i +  \sum_{i=1}^\numData
(1-\dataScalar_i)\left(\transformationFunction\left(\mappingVector^\top \designVector_i\right)\right)
\designVector_i.
$$ so the gradient with respect to one point is $$
\frac{\text{d}\errorFunction_i(\mappingVector)}{\text{d}\mappingVector}=\dataScalar_i\left(1-\transformationFunction\left(\mappingVector^\top \designVector_i\right)\right)
\designVector_i + (1-\dataScalar_i)\left(\transformationFunction\left(\mappingVector^\top \designVector_i\right)\right)
\designVector_i.
$$ and the stochastic gradient update for logistic regression (with
mini-batch size set to 1) is $$
\mappingVector_\text{new} \leftarrow \mappingVector_\text{old} - \eta ((1-\dataScalar_i)\left(\transformationFunction\left(\mappingVector^\top \designVector_i\right)\right) - \dataScalar_i\left(1-\transformationFunction\left(\mappingVector^\top \designVector_i\right)\right)
\designVector_i)
$$ The difference between the two is that for the perceptron we are
using the Heaviside function, $\heaviside(\cdot)$, whereas for logistic
regression we’re using the sigmoid, $\transformationFunction(\cdot)$,
which is like a soft version of the Heaviside function.

In [ ]:
import matplotlib.pyplot as plt
import mlai
import mlai.plot

In [ ]:
fig, ax = plt.subplots(figsize=plot.two_figsize)
f = np.linspace(-8, 8, 100)
h = 1/(1+np.exp(-f))
H = np.zeros(100)
H[51:] = 1.0
ax.plot(f, h, 'r-', lw=3)
ax.plot(f, H, 'b-', lw=3)

ax.set_title('Sigmoid vs Heaviside Function', fontsize=20)
ax.set_xlabel('$f_i$', fontsize=20)
ax.set_ylabel('$h_i$', fontsize=20)
mlai.write_figure('sigmoid-heaviside.svg', directory='\writeDiagrams/ml', transparent=True)


Because $\pi(\designVector_i)=\transformationFunction(\designVector_i)$,
when $\transformationFunction(\designVector_i)>0.5$ we classify as the
positive class, otherwise we classify as the negative class. So the
decision boundary is just like the peceptron. The vector
$\mappingVector$ gives the normal vector for the decision boundary.

In two dimensions, in the linear basis, the prediction function is $$
\transformationFunction\left(w_1 \inputScalar_{i, 1} + w_2 \inputScalar_{i, 2} + b\right)
$$ and the equation of a plane is $$
w_1 \inputScalar_{i, 1} + w_2 \inputScalar_{i, 2} + b = 0
$$ or $$
w_1 \inputScalar_{i, 1} + w_2 \inputScalar_{i, 2} = -b.
$$

We can mirror the way we implemented the Perceptron by selecting a
point, $i$, at random and setting the logistic regression weights to
that point, $\mappingVector = \designVector_i$, if it is in the positive
class, and setting to $\mappingVector = -\designVector_i$ if it is in
the negatvie class. That guarantees that the selected point will be
correctly classified.

In [ ]:
def init_logistic_regression(phi_plus, phi_minus, seed=1000001):
    """
    Initialise the perceptron algorithm with random weights and bias.

    The perceptron is a simple binary classifier that learns a linear decision boundary.
    This function initialises the weight vector w and bias b by randomly selecting
    a point from either the positive or negative class and setting the normal vector
    accordingly.

    Mathematical formulation:
    The perceptron decision function is f(x) = w^T φ, where:
    - w is the weight vector (normal to the decision boundary)
    - φ is the design matrix vector

    :param phi_plus: Positive class data points, shape (n_plus, n_features)
    :type phi_plus: numpy.ndarray
    :param phi_minus: Negative class data points, shape (n_minus, n_features)
    :type phi_minus: numpy.ndarray
    :param seed: Random seed for reproducible initialisation
    :type seed: int, optional
    :returns: Initial weight vector, shape (n_features,)
    :rtype: numpy.ndarray
    :returns: Initial bias term
    :rtype: float
    :returns: The randomly selected point used for initialisation
    :rtype: numpy.ndarray

    Examples:
        >>> phi_plus = np.array([[1, 1, 2], [1, 2, 3], [1, 3, 4]])
        >>> phi_minus = np.array([[1, 0, 0], [1, 1, 0], [1, 0, 1]])
        >>> w, phi_select = init_logistic_regression(phi_plus, phi_minus)
        >>> print(f"Weight vector: {w}")
    """
    np.random.seed(seed=seed)
    # flip a coin (i.e. generate a random number and check if it is greater than 0.5)
    plus_portion = phi_plus.shape[0]/(phi_plus.shape[0] + phi_minus.shape[0])
    choose_plus = np.random.rand(1)<plus_portion
    if choose_plus:
        # generate a random point from the positives
        index = np.random.randint(0, phi_plus.shape[0])
        phi_select = phi_plus[index, :]
        w = phi_plus[index, :].astype(float)  # set the normal vector to plus that point
    else:
        # generate a random point from the negatives
        index = np.random.randint(0, phi_minus.shape[0])
        phi_select = phi_minus[index, :]
        w = -phi_minus[index, :].astype(float)  # set the normal vector to minus that point.
    return w, phi_select

In [ ]:
def init_logistic_regression_plot(f, ax, phi_plus, phi_minus, w, fontsize=18):
    """
    Initialize a plot for showing the logistic decision boundary.

    :param f: Matplotlib figure object.
    :param ax: Array of matplotlib axes (should have 2 axes).
    :param phi_plus: Positive class design points (numpy array).
    :param phi_minus: Negative class design  points (numpy array).
    :param w: Weight vector for the logistic regression.
    :param fontsize: Font size for labels and titles (default: 18).
    :returns: Dictionary containing plot handles for updating.
    """
    h = {}

    ax[0].set_aspect('equal')
    # Plot the data again
    ax[0].plot(phi_plus[:, 1], phi_plus[:, 2], 'rx')
    ax[0].plot(phi_minus[:, 1], phi_minus[:, 2], 'go')
    plot_limits = {}
    plot_limits['x'] = np.asarray(ax[0].get_xlim())
    plot_limits['y'] = np.asarray(ax[0].get_ylim())
    x0, x1 = plot.hyperplane_coordinates(w[1:], w[0], plot_limits)
    strt = -w[0]/w[2]

    norm = w[1]*w[1] + w[2]*w[2]
    offset0 = -w[1]/norm*w[0]
    offset1 = -w[2]/norm*w[0]
    h['arrow'] = ax[0].arrow(offset0, offset1, offset0+w[1], offset1+w[2], head_width=0.2)
    # plot a line to represent the separating 'hyperplane'
    h['plane'], = ax[0].plot(x0, x1, 'b-')
    ax[0].set_xlim(plot_limits['x'])
    ax[0].set_ylim(plot_limits['y'])
    ax[0].set_xlabel('$x_0$', fontsize=fontsize)
    ax[0].set_ylabel('$x_1$', fontsize=fontsize)
    h['iter'] = ax[0].set_title('Update 0')

    bins = 15
    f_minus = np.dot(phi_minus, w)
    f_plus = np.dot(phi_plus, w)
    ax[1].hist(f_plus, bins, alpha=0.5, label='+1', color='r')
    ax[1].hist(f_minus, bins, alpha=0.5, label='-1', color='g')
    ax[1].legend(loc='upper right')
    return h


In [ ]:
import mlai.plot as plot

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(14,7))
w, phi_select = init_logistic_regression(phi_plus, phi_minus)
handle = init_logistic_regression_plot(f, ax, phi_plus, phi_minus, w)
mlai.write_figure("logistic-regression_init.svg", directory="./ml")

Learning then proceeds

In [ ]:
def logistic(x):
    """
    Compute the logistic function.
    """
    return 1/(1+np.exp(x))

def update_logistic_regression(w, phi_plus, phi_minus, learn_rate):
    """
    Update the logistic regression weights and bias using stochastic gradient descent.

    This function implements one step of the perceptron learning algorithm.
    It randomly selects a training point and updates the weights if the point
    is misclassified.

    Mathematical formulation:
    The perceptron update rule is:
    - If y_i = +1 and w^T φ_i ≤ 0: w ← w + η    h(w^Tφ_i) φ_i
    - If y_i = -1 and w^T φ_i > 0: w ← w - η (1-h(w^Tφ_i))φ_i

    where η is the learning rate.

    :param w: Current weight vector, shape (n_features,)
    :type w: numpy.ndarray
    :param phi_plus: Positive class design matrix, shape (n_plus, n_features)
    :type phi_plus: numpy.ndarray
    :param phi_minus: Negative class design matrix, shape (n_minus, n_features)
    :type phi_minus: numpy.ndarray
    :param learn_rate: Learning rate (step size) for weight updates
    :type learn_rate: float
    :returns: Updated weight vector
    :rtype: numpy.ndarray
    :returns: Updated bias term
    :rtype: float
    :returns: The randomly selected point used for the update
    :rtype: numpy.ndarray
    :returns: True if weights were updated, False otherwise
    :rtype: bool

    Examples:
        >>> w = np.array([0.1 0.5, -0.3])
        >>> phi_plus = np.array([[1, 1, 2], [1, 2, 3]])
        >>> phi_minus = np.array([[1, 0, 0], [1, 1, 0]])
        >>> w_new, phi_select, updated = update_perceptron(w, phi_plus, phi_minus, 0.1)
    """
    # select a point at random from the data
    plus_portion = phi_plus.shape[0]/(phi_plus.shape[0] + phi_minus.shape[0])
    choose_plus = np.random.rand(1)<plus_portion
    updated=False
    if choose_plus:
        # choose a point from the positive data
        index = np.random.randint(phi_plus.shape[0])
        phi_select = phi_plus[index, :]
        a = np.dot(w, phi_select)
        if np.dot(w, phi_select) <= 0.:
            # point is currently incorrectly classified
            w += learn_rate*logistic(a)*phi_select
            updated=True
    else:
        # choose a point from the negative data
        index = np.random.randint(phi_minus.shape[0])
        phi_select = phi_minus[index, :]
        a = np.dot(w, phi_select)
        if a > 0.:
            # point is currently incorrectly classified
            w -= learn_rate*(1-logistic(a))*phi_select
    return w, phi_select

In [ ]:
def update_logistic_plot(h, f, ax, phi_plus, phi_minus, i, w):
    """
    Update plots after decision boundary has changed.

    :param h: Dictionary containing plot handles from init_logistic_regression.
    :param f: Matplotlib figure object.
    :param ax: Array of matplotlib axes.
    :param phi_plus: Positive class data points.
    :param phi_minus: Negative class data points.
    :param i: Current iteration number.
    :param w: Updated weight vector.
    """
    # Re-plot the hyper plane
    plot_limits = {}
    plot_limits['x'] = np.asarray(ax[0].get_xlim())
    plot_limits['y'] = np.asarray(ax[0].get_ylim())
    x0, x1 = plot.hyperplane_coordinates(w[1:], w[0], plot_limits)

    # Add arrow to represent hyperplane.
    h['arrow'].remove()
    del(h['arrow'])
    norm = (w[1]*w[1] + w[2]*w[2])
    offset0 = -w[1]/norm*w[0]
    offset1 = -w[2]/norm*w[0]
    h['arrow'] = ax[0].arrow(offset0, offset1, offset0+w[1],
                             offset1+w[2], head_width=0.2)

    h['plane'].set_xdata(x0)
    h['plane'].set_ydata(x1)

    h['iter'].set_text('Iteration ' + str(i))
    ax[1].cla()
    bins = 15
    f_minus = np.dot(phi_minus, w)
    f_plus = np.dot(phi_plus, w)
    ax[1].hist(f_plus, bins, alpha=0.5, label='+1', color='r')
    ax[1].hist(f_minus, bins, alpha=0.5, label='-1', color='g')
    ax[1].legend(loc='upper right')

    IPython.display.display(f)
    IPython.display.clear_output(wait=True)
    return h



In [ ]:
import IPython
import mlai
import mlai.plot
import matplotlib.pyplot as plt

In [ ]:
diagrams = './ml'
seed = 42
max_iters = 30
learn_rate = 0.01
w, phi_select = init_logistic_regression(phi_plus, phi_minus, seed=seed)
count = 0
iterations = 0
setup=True
f2, ax2 = plt.subplots(1, 2, figsize=plot.two_figsize)
handle = init_logistic_regression_plot(f2, ax2, phi_plus, phi_minus, w)
handle['plane'].set_visible(False)
handle['arrow'].set_visible(False)
handle['circle'] = plt.Circle((phi_select[0], phi_select[1]), 0.25, color='b', fill=False)
ax2[0].add_artist(handle['circle'])
mlai.write_figure(figure=f2, filename='logistic-regression{samp:0>3}.svg'.format(samp=count), directory=diagrams, transparent=True)
extent = ax2[0].get_window_extent().transformed(f2.dpi_scale_trans.inverted())
mlai.write_figure(figure=f2, filename='logistic-regression{samp:0>3}.png'.format(samp=count), directory=diagrams, bbox_inches=extent, transparent=True)
count += 1
handle['plane'].set_visible(True)
handle['arrow'].set_visible(True)
mlai.write_figure(figure=f2, filename='logistic-regression{samp:0>3}.svg'.format(samp=count), directory=diagrams, transparent=True)
mlai.write_figure(figure=f2, filename='logistic-regression{samp:0>3}.png'.format(samp=count), directory=diagrams, bbox_inches=extent, transparent=True)

while iterations<max_iters:
    iterations += 1
    w, phi_select = update_logistic_regression(w, phi_plus, phi_minus, learn_rate)
    count+=1
    handle['circle'].center = phi_select[1], phi_select[2]
    mlai.write_figure(figure=f2, filename='logistic-regression{samp:0>3}.svg'.format(samp=count), directory=diagrams, transparent=True)
    mlai.write_figure(figure=f2, filename='logistic-regression{samp:0>3}.png'.format(samp=count), bbox_inches=extent, directory=diagrams, transparent=True)
    count+=1
    handle = update_logistic_plot(handle, f2, ax2, phi_plus, phi_minus, iterations, w)
    mlai.write_figure(filename='logistic-regression{samp:0>3}.svg'.format(samp=count),
                      figure=f2,
                      directory=diagrams,
                      transparent=True)
    mlai.write_figure(filename='logistic-regression{samp:0>3}.png'.format(samp=count),
                      figure=f2,
                      directory=diagrams,
                      bbox_inches=extent,
                      transparent=True)
print('Data passes:', iterations)


In [ ]:
import notutils as nu

In [ ]:
nu.display_plots('logistic-regression{samp:0>3}.svg', directory='./ml', samp=(0, count))

<img src="https://mlatcl.github.io/mlfc/./slides/diagrams//ml/logistic-regression014.svg" class="" width="60%" style="vertical-align:middle;">

Figure: <i>The logistic-regression decision boundary.</i>

–\>

    ::: {.cell .markdown}
    # Nigeria NMIS Data

    ::: {style="text-align:right"}
    [\[`</span>`{=html}[`<a href="https://github.com/lawrennd/snippets/edit/main/_datasets/includes/nigeria-nmis-data.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_datasets/includes/nigeria-nmis-data.md', 13);">`{=html}edit`</a>`{=html}]{.editsection
    style=""}`<span class="editsection-bracket" style="">`{=html}\]]{.editsection-bracket
    style=""}
    :::

    As an example data set we will use Nigerian Millennium Development Goals
    Information System Health Facility [@Nigeria-nmis14]. It can be found
    here
    <https://energydata.info/dataset/nigeria-nmis-education-facility-data-2014>.

    Taking from the information on the site,

    > The Nigeria MDG (Millennium Development Goals) Information System --
    > NMIS health facility data is collected by the Office of the Senior
    > Special Assistant to the President on the Millennium Development Goals
    > (OSSAP-MDGs) in partner with the Sustainable Engineering Lab at
    > Columbia University. A rigorous, geo-referenced baseline facility
    > inventory across Nigeria is created spanning from 2009 to 2011 with an
    > additional survey effort to increase coverage in 2014, to build
    > Nigeria's first nation-wide inventory of health facility. The database
    > includes 34,139 health facilities info in Nigeria.
    >
    > The goal of this database is to make the data collected available to
    > planners, government officials, and the public, to be used to make
    > strategic decisions for planning relevant interventions.
    >
    > For data inquiry, please contact Ms. Funlola Osinupebi, Performance
    > Monitoring & Communications, Advisory Power Team, Office of the Vice
    > President at funlola.osinupebi\@aptovp.org
    >
    > To learn more, please visit
    > <http://csd.columbia.edu/2014/03/10/the-nigeria-mdg-information-system-nmis-takes-open-data-further/>
    >
    > Suggested citation: Nigeria NMIS facility database (2014), the Office
    > of the Senior Special Assistant to the President on the Millennium
    > Development Goals (OSSAP-MDGs) & Columbia University

    For ease of use we've packaged this data set in the `pods` library
    :::

    ::: {.cell .code}
    ``` {.python}
    data = pods.datasets.nigeria_nmis()['Y']
    data.head()

:::

Alternatively, you can access the data directly with the following
commands.

``` python
import urllib.request
urllib.request.urlretrieve('https://energydata.info/dataset/f85d1796-e7f2-4630-be84-79420174e3bd/resource/6e640a13-cab4-457b-b9e6-0336051bac27/download/healthmopupandbaselinenmisfacility.csv', 'healthmopupandbaselinenmisfacility.csv')

import pandas as pd
data = pd.read_csv('healthmopupandbaselinenmisfacility.csv')
```

Once it is loaded in the data can be summarized using the `describe`
method in pandas.

In [ ]:
data.describe()

We can also find out the dimensions of the dataset using the `shape`
property.

In [ ]:
data.shape

Dataframes have different functions that you can use to explore and
understand your data. In python and the Jupyter notebook it is possible
to see a list of all possible functions and attributes by typing the
name of the object followed by `.<Tab>` for example in the above case if
we type `data.<Tab>` it show the columns available (these are attributes
in pandas dataframes) such as `num_nurses_fulltime`, and also functions,
such as `.describe()`.

For functions we can also see the documentation about the function by
following the name with a question mark. This will open a box with
documentation at the bottom which can be closed with the x button.

In [ ]:
data.describe?

In [ ]:
import matplotlib.pyplot as plt
import mlai
import mlai.plot as plot

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_figsize)
ax.plot(data.longitude, data.latitude, 'ro', alpha=0.01)
ax.set_xlabel('longitude')
ax.set_ylabel('latitude')

mlai.write_figure('nigerian-health-facilities.png', directory='./ml')

<img class="" src="https://mlatcl.github.io/mlfc/./slides/diagrams//ml/nigerian-health-facilities.png" style="width:60%">

Figure: <i>Location of the over thirty-four thousand health facilities
registered in the NMIS data across Nigeria. Each facility plotted
according to its latitude and longitude.</i>

## Nigeria NMIS Data Classification

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_datasets/includes/nigeria-nmis-data-classification.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_datasets/includes/nigeria-nmis-data-classification.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

Our aim will be to predict whether a center has maternal health delivery
services given the attributes in the data. We will predict of the number
of nurses, the number of doctors, location etc.

Now we will convert this data into a form which we can use as inputs
`X`, and labels `y`.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = data[~pd.isnull(data['maternal_health_delivery_services'])]
data = data.dropna() # Remove entries with missing values
X = data[['emergency_transport',
          'num_chews_fulltime',
          'phcn_electricity',
          'child_health_measles_immun_calc',
          'num_nurses_fulltime',
          'num_doctors_fulltime',
          'improved_water_supply',
          'improved_sanitation',
          'antenatal_care_yn',
          'family_planning_yn',
          'malaria_treatment_artemisinin',
          'latitude',
          'longitude']].copy()
y = data['maternal_health_delivery_services']==True  # set label to be whether there's a maternal health delivery service

# Create series of health center types with the relevant index
s = data['facility_type_display'].apply(pd.Series).stack()
s.index = s.index.droplevel(-1) # to line up with df's index

# Extract from the series the unique list of types.
types = s.unique()

# For each type extract the indices where it is present and add a column to X
type_names = []
for htype in types:
    index = s[s==htype].index.tolist()
    type_col=htype.replace(' ', '_').replace('/','-').lower()
    type_names.append(type_col)
    X.loc[:, type_col] = 0.0
    X.loc[index, type_col] = 1.0

This has given us a new data frame `X` which contains the different
facility types in different columns.

In [ ]:
X.describe()

## Batch Gradient Descent

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_ml/includes/nigeria-nmis-data-logistic.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_ml/includes/nigeria-nmis-data-logistic.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

We will need to define some initial random values for our vector and
then minimize the objective by descending the gradient.

In [ ]:
# Separate train and test
indices = np.random.permutation(X.shape[0])
num_train = int(np.ceil(X.shape[0]/2))
train_indices = indices[:num_train]
test_indices = indices[num_train:]
X_train = X.iloc[train_indices]
y_train = y.iloc[train_indices]==True
X_test = X.iloc[test_indices]
y_test = y.iloc[test_indices]==True

In [ ]:
import numpy as np

In [ ]:
# gradient descent algorithm
w_vals = np.random.normal(size=(X.shape[1]+1, 1), scale = 0.001)
# Convert weights to a pandas DataFrame with column names
if type(X_train) is pd.DataFrame:
    w = pd.DataFrame(data=w_vals,
                     index=['Eins'] + list(X_train.columns),
                     columns=['weight'])

eta = 1e-9
iters = 10000
for i in range(iters):
    g, Phi = predict(w, X_train, mlai.linear)
    w -= eta*gradient(g, Phi, y_train) + 0.001*w
    if not i % 100:
        print("Iter", i, "Objective", objective(g, y_train))

Let’s look at the weights and how they relate to the inputs.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Plot a histogram of the weights
plt.figure(figsize=(10, 6))
if type(w) is pd.DataFrame:
    plt.bar(w.index, w['weight'])
else:
    plt.bar(w)
plt.xticks(rotation=90)
plt.xlabel('Features')
plt.ylabel('Weight Value')
plt.title('Histogram of Model Weights')
plt.tight_layout()
mlai.write_figure('nmis-logistic-regression-weights.svg', directory='./ml')

### Exercise 1

What does the magnitude of the weight vectors tell you about the
different parameters and their influence on outcome? Are the weights of
roughly the same size, if not, how might you fix this?

### Exercise 1 Answer

Write your answer to Exercise 1 here

In [ ]:
g_test, Phi_test = predict(w, X_test, mlai.linear)
np.sum(g_test[y_test]>0.5)

## Stochastic Gradient Descent

### Exercise 2

Now construct a stochastic gradient descent algorithm and run it on the
data. Is it faster or slower than batch gradient descent? What can you
do to improve convergence speed?

In [ ]:
# Write your answer to Exercise 2 here






In [ ]:
%pip install statsmodels

## Linear Regression with `statsmodels`

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_statistics/includes/linear-regression-statsmodels.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_statistics/includes/linear-regression-statsmodels.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

In linear regression, we model the relationship between a continuous
response variable $y_i$ and input variables $\mathbf{ x}_i$ through a
linear function with Gaussian noise:

$$y_i = f(\mathbf{ x}_i) + \epsilon_i$$

where
$f(\mathbf{ x}_i) = \mathbf{ w}^\top\mathbf{ x}_i = \sum_{j=1}^D w_jx_{i,j}$
and $\epsilon_i \sim \mathscr{N}\left(0,\sigma^2\right)$

This gives us a probabilistic model:

$$p(y_i|\mathbf{ x}_i) = \mathscr{N}\left(y_i|\mathbf{ w}^\top\mathbf{ x}_i,\sigma^2\right)$$

The key components are:

-   $y_i$ is the target/response variable we want to predict
-   $\mathbf{ x}_i$ contains the input features/explanatory variables  
-   $\mathbf{ w}$ contains the parameters/coefficients we learn
-   $\epsilon_i$ represents random Gaussian noise with variance
    $\sigma^2$

For the full dataset, we can write this in matrix form:

$$\mathbf{ y}= \mathbf{X}\mathbf{ w}+ \boldsymbol{ \epsilon}$$

where $\mathbf{ y}= [y_1,\ldots,y_N]^\top$, $\mathbf{X}$ contains the
input vectors as rows, and
$\boldsymbol{ \epsilon}\sim \mathscr{N}\left(\mathbf{0},\sigma^2\mathbf{I}\right)$.

The expected value of our prediction is:

$$\mathbb{E}[y_i|\mathbf{ x}_i] = \mathbf{ w}^\top\mathbf{ x}_i$$

This linear model forms the foundation for generalized linear models
like logistic regression, where we’ll adapt the model for classification
by transforming the output through a non-linear function.

In [ ]:
import statsmodels.api as sm
import pods

In [ ]:
# Demo of linear regression using python statsmodels.
data = pods.datasets.olympic_marathon_men()
x = data['X']
y = data['Y']
# Add constant term to design matrix
x = sm.add_constant(x)
model = sm.OLS(y, x)
results = model.fit()
results.summary()

The statsmodels summary provides several key diagnostic measures that
help us evaluate our model fit and identify potential areas for
improvement. Since we’re working with one-dimensional data (year vs
time), we can visualize everything easily to complement these
statistical measures.

The model fit statistics show a moderately strong fit, with an R-squared
of 0.730 indicating that our model explains 73.0% of the variance in the
data. The adjusted R-squared of 0.721 confirms this isn’t just due to
overfitting. The very low F-statistic p-value (1.85e-09) confirms the
model’s overall significance. The AIC (11.33) and BIC (14.13) values
will be useful when we compare this model against alternative
specifications we might try.

Looking at the model parameters, we see a coefficient of -0.0116 for our
predictor, with a small standard error (0.001). The t-statistic of
-8.710 and p-value of 0.000 indicate this effect is highly significant.
The 95% confidence interval \[-0.014, -0.009\] gives us good confidence
in our estimate. The negative coefficient confirms the expected downward
trend in marathon times over the years.

However, the residual diagnostics suggest several potential issues we
should investigate:

1.  The Durbin-Watson statistic (0.981) indicates positive
    autocorrelation in the residuals. This suggests we might want to:

    -   Consider time series modeling approaches
    -   Add polynomial terms to capture non-linear trends
    -   Investigate if there are distinct “eras” in marathon times

2.  The highly significant Jarque-Bera test (p-value 8.32e-14) tells us
    our residuals aren’t normally distributed. The skew (1.947) and
    kurtosis (8.746) values show the distribution is strongly
    right-skewed with very heavy tails. We might want to:

    -   Look for outliers or influential points
    -   Consider robust regression techniques
    -   Try transforming our response variable

3.  The large condition number (9.94e+04) suggests potential numerical
    instability or multicollinearity issues. While less concerning with
    single-predictor models, we should:

    -   Consider centering and scaling our predictor
    -   Watch for numerical precision issues
    -   Be cautious when extending to multiple predictors

The beauty of having one-dimensional data is that we can plot everything
to visually confirm these statistical findings. A scatter plot with our
fitted line will help us:

-   Visually assess the linearity assumption
-   Identify potential outliers
-   Spot any systematic patterns in the residuals
-   See if the relationship makes practical sense in terms of marathon
    performance over time

This visual inspection, combined with our statistical diagnostics, will
guide our next steps in improving the model.

In [ ]:
import matplotlib.pyplot as plt
import mlai
from mlai import plot

In [ ]:
fig, ax = plt.subplots(figsize=mlai.plot.big_wide_figsize)
ax.plot(x[:, 1], y, '.')

# Plot the fitted line
ax.plot(x[:, 1], results.predict(x), '-')

ax.set_xlabel('Year')
ax.set_ylabel('Time')
ax.set_xlim(1890, 2030)
plt.show()
mlai.write_figure("linear-regression-olympic-marathon-men-statsmodels.svg", directory="./data-science")

Looking at our plot and model diagnostics, we can now better understand
the large condition number (1.08e+05) in our results. This high value
likely stems from using raw year values (e.g., 1896, 1900, etc.) as our
predictor variable. Such large numbers can lead to numerical instability
in the computations.

To address this, we could consider:

-   Centering the years around their mean
-   Scaling the years to a smaller range (e.g., 0-1)
-   Using years since the first Olympics (e.g., 0, 4, 8, etc.)

Any of these transformations would help reduce the condition number
while preserving the underlying relationship in our data. The
coefficients would change, but the fitted values and overall model
quality would remain the same.

<img src="https://mlatcl.github.io/mlfc/./slides/diagrams//data-science/linear-regression-olympic-marathon-men-statsmodels.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Linear regression fit to Olympic marathon men’s times using
`statsmodels`.</i>

The plot reveals features that help explain our diagnostic statistics:

-   The 1904 St. Louis Olympics appears as a clear outlier, contributing
    to the non-normal residuals (Jarque-Bera p=0.00432) and right-skewed
    distribution (skew=1.385)
-   We can observe distinct regimes in the data:
    -   Rapid improvement in times pre-WWI
    -   Disruption and variation during the war years
    -   More steady, consistent progress post-WWII
-   These regime changes help explain the strong positive
    autocorrelation (Durbin-Watson=0.242) in our residuals
-   While our high R-squared (0.972) captures the overall downward
    trend, these features suggest we could improve the model by adding
    additional features:
    -   Polynomial terms to capture non-linear trends
    -   Indicator variables for different time periods
    -   Interaction terms between features
    -   Variables accounting for external factors like temperature or
        course conditions

To incorporate multiple features into our model, we need a systematic
way to organize this additional information. This brings us to the
concept of the design matrix.

### Design Matrix

The design matrix, often denoted as $\boldsymbol{ \Phi}$, is a key
component of a statistical model. It organizes our feature data in a
structured way that facilitates model fitting and analysis. Each row of
the design matrix represents a single observation or data point, while
each column represents a different feature or predictor variable.

For $n$ observations and $p$ features, the design matrix takes the form:

$$\boldsymbol{ \Phi}= \begin{bmatrix}
x_{11} & x_{12} & \cdots & x_{1p} \\
x_{21} & x_{22} & \cdots & x_{2p} \\
\vdots & \vdots & \ddots & \vdots \\
x_{n1} & x_{n2} & \cdots & x_{np}
\end{bmatrix}$$

For example, if we’re predicting house prices, each row might represent
a different house, with columns for features like:

-   Square footage
-   Number of bedrooms  
-   Year built
-   Lot size

The design matrix provides a compact way to represent all our feature
data and is used directly in model fitting. When we write our linear
model as
$\mathbf{ y}= \boldsymbol{ \Phi}\mathbf{ w}+ \boldsymbol{ \epsilon}$,
the design matrix $\boldsymbol{ \Phi}$ is multiplied by our parameter
vector $\mathbf{ w}$ to generate predictions.

The design matrix often includes a column of 1s to account for the
intercept term in our model. This allows us to write the model in matrix
form without explicitly separating out the intercept term.

In [ ]:
import statsmodels.api as sm
import pods
import numpy as np

In [ ]:
# Demo of additional features with interactions regression usying python statsmodels.
data = pods.datasets.olympic_marathon_men()
x = data['X']
y = data['Y']

# Scale the year to avoid numerical issues
x_scaled = (x - 1900) / 100  # Center around 1900 and scale to century units

# Add to design matrix indicator variable for pre-1914
x_aug = np.hstack([x_scaled, (x[:, 0] < 1914).astype(np.float64)[:, np.newaxis]])

# Add to design matrix indicator variable for 1914-1945
x_aug = np.hstack([x_aug, ((x[:, 0] >= 1914) & (x[:, 0] <= 1945)).astype(np.float64)[:, np.newaxis]])

# Add to design matrix indicator variable for post-1945
x_aug = np.hstack([x_aug, (x[:, 0] > 1945).astype(np.float64)[:, np.newaxis]])

# Add product terms that multiply the scaled year and the indicator variables.
x_aug = np.hstack([x_aug, x_scaled[:, 0:1] * x_aug[:, 1:2], x_scaled[:, 0:1] * x_aug[:, 2:3]])

# Add constant term to design matrix
x_aug = sm.add_constant(x_aug)

# Do the linear fit
model = sm.OLS(y, x_aug)
results = model.fit()
results.summary()

In [ ]:
import matplotlib.pyplot as plt
import mlai
from mlai import plot

In [ ]:
fig, ax = plt.subplots(figsize=mlai.plot.big_wide_figsize)
ax.plot(x[:, 0], y, '.')

# Plot the fitted line
ax.plot(x[:, 0], results.predict(x_aug), '-')

ax.set_xlabel('Year')
ax.set_ylabel('Time')
ax.set_xlim(1890, 2030)
plt.show()
mlai.write_figure("linear-regression-olympic-marathon-men-augmented-statsmodels.svg", directory="./data-science")

<img src="https://mlatcl.github.io/mlfc/./slides/diagrams//data-science/linear-regression-olympic-marathon-men-augmented-statsmodels.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Polynomial regression fit to Olympic marathon men’s times
using `statsmodels`.</i>

The augmented model with interactions shows a significant improvement in
fit compared to the simpler linear model, with an R-squared value of
0.877 (adjusted R-squared of 0.851). This indicates that about 87% of
the variance in marathon times is explained by our model.

The model includes several key components:

-   A base time trend (x1 coefficient: -0.5634)
-   Indicator variables for different historical periods (pre-1914,
    1914-1945, post-1945)
-   Interaction terms between the time trend and these periods

The coefficients reveal interesting patterns:

-   The pre-1914 period shows a significant positive effect (x2: 1.5700,
    p\<0.001)
-   The wartime period 1914-1945 also shows a positive effect (x3:
    0.8176, p=0.030)
-   The post-1945 period has a positive effect (x4: 0.6300, p=0.002)
-   The interaction terms (x5, x6) suggest different rates of
    improvement in different periods, though these are less
    statistically significant (x5: -3.0493, p=0.076; x6: -0.1694,
    p=0.919)

However, there are some concerns:

1.  The very high condition number (2.17e+16) suggests serious
    multicollinearity issues
2.  The highly significant Jarque-Bera test (p-value 1.34e-24) indicates
    non-normal residuals
3.  There’s significant skewness (2.393) and kurtosis (11.065) in the
    residuals

Despite these statistical issues, the model captures the major trends in
marathon times across different historical periods better than a simple
linear regression would.

## Fitting with `statsmodels`

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_statistics/includes/logistic-regression-statsmodels.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_statistics/includes/logistic-regression-statsmodels.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

## Fitting the Model

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_statistics/includes/logistic-regression-train.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_statistics/includes/logistic-regression-train.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

In [ ]:
# Demo of logistic regression using python statsmodels.
# Create a synthetic binary classification dataset
X, y = make_classification(n_samples=200, n_features=2, n_redundant=0,
                         n_informative=2, n_clusters_per_class=1,
                         random_state=42)

# Convert to DataFrame for easier handling
df = pd.DataFrame(X, columns=['feature1', 'feature2'])
df['target'] = y

# Split into train and test sets
indices = np.random.permutation(df.shape[0])
num_train = int(np.ceil(df.shape[0]/2))
train_indices = indices[:num_train]
test_indices = indices[num_train:]

X_train = df[['feature1', 'feature2']].iloc[train_indices]
y_train = df['target'].iloc[train_indices]
X_test = df[['feature1', 'feature2']].iloc[test_indices]
y_test = df['target'].iloc[test_indices]

# Add constant term to design matrix
X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)

# Fit logistic regression model
model = sm.Logit(y_train, X_train_sm)
results = model.fit()
results.summary()

## Model Fit Statistics

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_statistics/includes/logistic-regression-fit.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_statistics/includes/logistic-regression-fit.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

The statsmodels summary for logistic regression provides several
diagnostic measures that help us evaluate our classification model’s
performance and identify potential areas for improvement.

**Model Fit Statistics:** The logistic regression model doesn’t have a
traditional R-squared since we’re dealing with binary outcomes rather
than continuous responses. Instead, we use pseudo R-squared measures:

-   **McFadden’s R-squared**: Compares the log-likelihood of our model
    to a null model with only an intercept. Values between 0.2-0.4
    indicate excellent fit.
-   **Log-Likelihood Ratio (LLR) test**: Tests whether our model is
    significantly better than the null model. A low p-value indicates
    our predictors significantly improve the model.
-   **AIC/BIC**: Help compare different model specifications. Lower
    values indicate better models when comparing alternatives.

**Parameter Interpretation:** The coefficients in logistic regression
represent changes in log-odds: - A positive coefficient means the
feature increases the odds of the positive class - A negative
coefficient means the feature decreases the odds of the positive class  
- The magnitude indicates the strength of the effect - To get odds
ratios, we exponentiate the coefficients: $\exp(\beta_j)$

For example, if a coefficient is 0.693, then $\exp(0.693) = 2.0$,
meaning a one-unit increase in that feature doubles the odds of the
positive outcome.

**Diagnostic Considerations:** Unlike linear regression, logistic
regression has different diagnostic concerns:

1.  **Multicollinearity**: Check condition numbers and correlation
    matrices, just like in linear regression
2.  **Outliers and Influential Points**: Use deviance residuals and
    leverage measures to identify problematic observations
3.  **Model Adequacy**: Hosmer-Lemeshow test checks if predicted
    probabilities match observed frequencies
4.  **Separation**: Perfect or quasi-perfect separation can cause
    convergence issues and inflated standard errors

**Classification Performance:** Beyond the statistical diagnostics, we
should evaluate practical classification performance: - **Confusion
Matrix**: Shows true vs predicted classifications - **Accuracy**:
Overall percentage of correct predictions  
- **Precision/Recall**: Important when classes are imbalanced -
**ROC/AUC**: Measures discrimination ability across different thresholds

## Predictions of the Model

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_statistics/includes/logistic-regression-test.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_statistics/includes/logistic-regression-test.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

In [ ]:
# Make predictions on test set
y_pred_proba = results.predict(X_test_sm)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate classification metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Test Accuracy: {accuracy:.3f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"\nClassification Report:\n{classification_report(y_test, y_pred)}")

In [ ]:
import matplotlib.pyplot as plt
import mlai
from mlai import plot

In [ ]:
# Create visualization of the logistic regression results
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Data points colored by true class
ax1 = axes[0]
scatter = ax1.scatter(X_train['feature1'], X_train['feature2'],
                     c=y_train, cmap='RdYlBu', alpha=0.7, s=50)
ax1.set_xlabel('Feature 1')
ax1.set_ylabel('Feature 2')
ax1.set_title('Training Data (True Classes)')
plt.colorbar(scatter, ax=ax1)

# Plot 2: Decision boundary and predicted probabilities
ax2 = axes[1]

# Create a mesh to plot the decision boundary
h = 0.1
x_min, x_max = X_train['feature1'].min() - 1, X_train['feature1'].max() + 1
y_min, y_max = X_train['feature2'].min() - 1, X_train['feature2'].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# Predict probabilities on the mesh
mesh_points = np.c_[xx.ravel(), yy.ravel()]
mesh_points_sm = sm.add_constant(mesh_points)
Z = results.predict(mesh_points_sm)
Z = Z.reshape(xx.shape)

# Plot decision boundary and probability contours
contour = ax2.contourf(xx, yy, Z, levels=50, alpha=0.6, cmap='RdYlBu')
ax2.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2, linestyles='--')

# Plot training points
ax2.scatter(X_train['feature1'], X_train['feature2'],
           c=y_train, cmap='RdYlBu', edgecolors='black', s=50)
ax2.set_xlabel('Feature 1')
ax2.set_ylabel('Feature 2')
ax2.set_title('Decision Boundary and Probability Contours')
plt.colorbar(contour, ax=ax2)

plt.tight_layout()
plt.show()
mlai.write_figure("logistic-regression-classification-statsmodels.svg", directory="./statistics")

<img src="https://mlatcl.github.io/mlfc/./slides/diagrams//statistics/logistic-regression-classification-statsmodels.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Plot of the logistic regression fit made by
`statsmodels`.</i>

Looking at our classification results, we can evaluate several aspects
of model performance:

**Decision Boundary Analysis:** The visualization shows how our logistic
regression model creates a linear decision boundary in the feature
space. The dashed black line represents the 0.5 probability threshold
where the model switches between predicting class 0 and class 1. The
colored contours show the predicted probability landscape - areas closer
to red have higher probability of being class 1, while areas closer to
blue have higher probability of being class 0.

**Model Performance:** From the classification metrics, we can assess: -
**Accuracy**: Overall percentage of correct predictions on the test set
- **Confusion Matrix**: Shows the breakdown of true positives, false
positives, true negatives, and false negatives - **Precision and
Recall**: Important when we care about specific types of errors (e.g.,
medical diagnosis)

**Potential Issues to Monitor:** 1. **Feature Scaling**: If features
have very different scales, consider standardization 2. **Linear
Separability**: Our model assumes a linear decision boundary - if
classes aren’t linearly separable, consider polynomial features or
non-linear methods 3. **Class Imbalance**: If one class dominates,
consider resampling techniques or adjusting the decision threshold 4.
**Overfitting**: Monitor performance on validation data, especially with
many features

<img src="https://mlatcl.github.io/mlfc/./slides/diagrams//ml/logistic-regression-classification-statsmodels.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Logistic regression classification results showing training
data and decision boundary with probability contours using
`statsmodels`.</i>

The classification visualization reveals several important aspects of
our logistic regression model:

**Linear Decision Boundary:** The model creates a straight-line decision
boundary (shown as the dashed line at 0.5 probability). This linear
separator works well when classes are roughly linearly separable, but
may struggle with more complex class distributions.

**Probability Gradients:** The colored contours show how predicted
probabilities change smoothly across the feature space. Points far from
the decision boundary have probabilities close to 0 or 1 (high
confidence), while points near the boundary have probabilities around
0.5 (uncertain predictions).

**Model Extensions:** For more complex classification problems, we can
enhance the basic logistic regression model: - **Polynomial Features**:
Add $x_1^2$, $x_2^2$, $x_1 x_2$ terms for non-linear decision boundaries
- **Feature Interactions**: Include products of features to capture
synergistic effects - **Regularization**: Add L1 (Lasso) or L2 (Ridge)
penalties to prevent overfitting - **Feature Engineering**: Transform or
combine features to better capture relationships

To incorporate multiple features and transformations into our model, we
need a systematic way to organize this information through the design
matrix.

## Categorical (Multinomial) Regression

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_ml/includes/categorical-regression-intro.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_ml/includes/categorical-regression-intro.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

This is the multi-class analogue of logistic regression. Instead of a
single Bernoulli outcome, we model a $K$-way categorical outcome using a
softmax link over class-specific linear scores.

## Soft Arg Max (Softmax) Function

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_ml/includes/soft-arg-max-function.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_ml/includes/soft-arg-max-function.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

Given class scores
$f_k(\mathbf{ x})=\mathbf{ w}_k^\top\boldsymbol{ \phi}(\mathbf{ x})$,
the softmax maps scores to probabilities:
$\pi_k = \frac{\exp(f_k)}{\sum_{j=1}^K \exp(f_j)}$.

## Prediction Function

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_ml/includes/categorical-regression-prediction-function.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_ml/includes/categorical-regression-prediction-function.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

For class $k\in\{1,\dots,K\}$, define a linear score
$f_k(\mathbf{ x})=\mathbf{ w}_k^\top\boldsymbol{ \phi}(\mathbf{ x})$.
The predictive probabilities are given by the softmax: $$
\pi_k(\mathbf{ x}) = \frac{\exp\left(f_k(\mathbf{ x})\right)}{\sum_{j=1}^K \exp\left(f_j(\mathbf{ x})\right)}.
$$

## Maximum Likelihood

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_ml/includes/categorical-regression-maximum-likelihood.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_ml/includes/categorical-regression-maximum-likelihood.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

With one-hot targets $\mathbf{ y}_i\in\{0,1\}^K$ and softmax
probabilities $\pi_{ik}$, the (negative) log-likelihood is $$
E(\{\mathbf{ w}_k\}) = -\sum_{i=1}^n\sum_{k=1}^K y_{ik}\log \pi_{ik}.
$$

## Other GLMs

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_statistics/includes/other-glms-statsmodels.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_statistics/includes/other-glms-statsmodels.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

We’ve introduced the formalism for generalised linear models. Have a
think about how you might model count data using the [Poisson
distribution](http://en.wikipedia.org/wiki/Poisson_distribution) and a
log link function for the rate, $\lambda(\mathbf{ x})$. If you want a
data set you can try the `pods.datasets.google_trends()` for some count
data.

## Other GLMs

We’ve introduced the formalism for generalised linear models. Have a
think about how you might model count data using the [Poisson
distribution](http://en.wikipedia.org/wiki/Poisson_distribution) and a
log link function for the rate, $\lambda(\mathbf{ x})$. If you want a
data set you can try the `pods.datasets.google_trends()` for some count
data.

## Poisson Distribution

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_ml/includes/poisson-distribution.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_ml/includes/poisson-distribution.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

In [ ]:
import mlai.plot as plot

In [ ]:
plot.poisson('./ml/')

<img src="https://mlatcl.github.io/mlfc/./slides/diagrams//ml/poisson.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>The Poisson distribution.</i>

## Poisson Regression

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_ml/includes/poisson-regression.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_ml/includes/poisson-regression.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

Poisson regression is a type of generalized linear model (GLM) used when
modeling count data. It assumes the response variable follows a Poisson
distribution and uses a logarithmic link function to relate the mean of
the response to the linear predictor.

In this model, we make the rate parameter λ a function of covariates
(like space or time). The logarithm of the rate is modeled as a linear
combination of the input features:

$$\log \lambda(\mathbf{ x}, t) = \mathbf{ w}_x^\top \boldsymbol{ \phi}_x(\mathbf{ x}) + \mathbf{ w}_t^\top \boldsymbol{ \phi}_t(t)$$

where: - $\mathbf{ w}_x$ and $\mathbf{ w}_t$ are parameter vectors -
$\boldsymbol{ \phi}_x(\mathbf{ x})$ and $\boldsymbol{ \phi}_t(t)$ are
basis functions for space and time respectively

This formulation is known as a log-linear or log-additive model because
we’re adding terms in the log space. The logarithm serves as our link
function, connecting the linear predictor to the response variable’s
mean.

An important characteristic of this model that practitioners should be
aware of is that while we add terms in the log space, the model becomes
multiplicative when we transform back to the original space. This
happens because:

1.  We start with the log-additive form:
    $\log \lambda(\mathbf{ x}, t) = f_x(\mathbf{ x}) + f_t(t)$

2.  When we exponentiate both sides to get back to λ, the addition in
    log space becomes multiplication:
    $$\lambda(\mathbf{ x}, t) = \exp(f_x(\mathbf{ x}) + f_t(t)) = \exp(f_x(\mathbf{ x}))\exp(f_t(t))$$

This multiplicative nature has important implications for
interpretation. For example, if we increase one input variable, it has a
multiplicative effect on the rate, not an additive one. This can lead to
rapid growth in the predicted counts as input values increase.

Let’s look at another example using synthetic data to demonstrate
Poisson regression without relying on external APIs.

In [ ]:
import numpy as np
import statsmodels.api as sm

In [ ]:
# Generate some example count data
np.random.seed(42)
n_samples = 100
x1 = np.random.uniform(0, 10, n_samples)
x2 = np.random.uniform(0, 5, n_samples)
X = np.column_stack((x1, x2))

# True relationship: y ~ Poisson(exp(1 + 0.3*x1 - 0.2*x2))
lambda_true = np.exp(1 + 0.3*x1 - 0.2*x2)
y = np.random.poisson(lambda_true)

# Fit Poisson regression
model_synthetic = sm.GLM(y, sm.add_constant(X), family=sm.families.Poisson())
result_synthetic = model_synthetic.fit()

The `statsmodels` library in Python provides a convenient way to fit
Poisson regression models. The `sm.GLM` function is used to fit
generalized linear models, and we specify the Poisson family to indicate
that we’re modeling count data. The `sm.add_constant(x)` function adds a
column of ones to the design matrix to account for the intercept term.

In this synthetic example, we generate count data that follows a Poisson
distribution where the rate parameter $\lambda$ depends on two predictor
variables. This demonstrates how Poisson regression can model count data
with multiple predictors.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot actual vs predicted counts
y_pred = result_synthetic.predict(sm.add_constant(X))
ax1.scatter(y, y_pred, alpha=0.5)
ax1.plot([0, max(y)], [0, max(y)], 'r--')
ax1.set_xlabel('Actual Counts')
ax1.set_ylabel('Predicted Counts')
ax1.set_title('Actual vs Predicted')

# Plot residuals
residuals = y - y_pred
ax2.scatter(y_pred, residuals, alpha=0.5)
ax2.axhline(y=0, color='r', linestyle='--')
ax2.set_xlabel('Predicted Counts')
ax2.set_ylabel('Residuals')
ax2.set_title('Residual Plot')

plt.tight_layout()
plt.show()
mlai.write_figure("poisson-regression-diagnostics.svg", directory="./ml/")

<img src="https://mlatcl.github.io/mlfc/./slides/diagrams//ml/poisson-regression-diagnostics.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Diagnostic plots for the Poisson regression model showing
actual vs predicted counts and residual analysis.</i>

## Practical Tips

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_statistics/includes/glm-practical-tips.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_statistics/includes/glm-practical-tips.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

When working with generalised linear models in practice, there are
several key considerations that can significantly impact model
performance:

Feature engineering is often the most critical factor in model success:

-   Build modular data processing pipelines that allow you to easily
    test different feature sets. For example, if modeling house prices,
    you might want to test combinations of raw features (square footage,
    bedrooms), derived features (price per square foot), and interaction
    terms (bedrooms × bathrooms).
-   Consider non-linear transformations of continuous variables. For
    instance, taking the log of price data often helps normalize
    distributions.
-   Be thoughtful about encoding categorical variables - one-hot
    encoding isn’t always optimal. For high-cardinality categories,
    consider target encoding or feature hashing.
-   Scale features appropriately - standardization or min-max scaling
    depending on your model assumptions.
-   Document your feature creation process thoroughly, including the
    rationale for each transformation.

Model validation requires careful consideration:

-   Cross-validation should match your real-world use case. For time
    series data, use time-based splits rather than random splits.
-   Bootstrap sampling helps understand parameter uncertainty. For
    example, bootstrapping can show if a coefficient’s sign might flip
    under different samples.
-   Hold-out test sets should be truly independent. In a customer churn
    model, this might mean testing on future customers rather than a
    random subset.
-   Watch for data leakage, especially with time-dependent features. If
    predicting customer churn, using future purchase data would create
    leakage.

Diagnostic checks are essential for model reliability:

-   Create residual plots against fitted values and each predictor. Look
    for systematic patterns - a U-shaped residual plot suggests missing
    quadratic terms.
-   For logistic regression, plot predicted probabilities against actual
    outcomes in bins to check calibration.
-   Calculate influence measures like Cook’s distance to identify
    outliers. In a house price model, a mansion might have outsized
    influence on coefficients.
-   Check Variance Inflation Factors (VIF) for multicollinearity. High
    VIF (\>5-10) suggests problematic correlation between predictors.

Visualization remains crucial throughout:

-   Before modeling, create scatter plots, box plots, and histograms to
    understand your data distribution and relationships.
-   Use pairs plots to identify correlations and potential interactions
    between features.
-   Create residual diagnostic plots including Q-Q plots for normality
    checking.
-   When communicating results, focus on interpretable visualizations.
    For instance, partial dependence plots can show how predictions
    change with a single feature.

Additional practical considerations:

-   Start simple and add complexity incrementally. A basic linear model
    often provides a good baseline.
-   Keep track of model performance metrics across iterations to ensure
    changes actually improve results.
-   Consider the computational cost of feature engineering - some
    transformations might not be feasible in production.
-   Think about how features will be available in production. If a
    feature requires complex processing or external data, it might not
    be practical.
-   For categorical variables with many levels, consider grouping rare
    categories.
-   When dealing with missing data, document your imputation strategy
    and test its impact on model performance.

## Further Reading

-   Section 5.2.2 up to pg 182 of Rogers and Girolami (2011)

## Thanks!

For more information on these subjects and more you might want to check
the following resources.

-   company: [Trent AI](https://trent.ai)
-   book: [The Atomic
    Human](https://www.penguin.co.uk/books/455130/the-atomic-human-by-lawrence-neil-d/9780241625248)
-   twitter: [@lawrennd](https://twitter.com/lawrennd)
-   podcast: [The Talking Machines](http://thetalkingmachines.com)
-   newspaper: [Guardian Profile
    Page](http://www.theguardian.com/profile/neil-lawrence)
-   blog:
    [http://inverseprobability.com](http://inverseprobability.com/blog.html)

::: {.cell .markdown}

## References

Robbins, H., Monro, S., 1951. A stochastic approximation method. Annals
of Mathematical Statistics 22, 400–407.

Rogers, S., Girolami, M., 2011. A first course in machine learning. CRC
Press.